In [1]:
from datasets import load_dataset

dataset = load_dataset("NTTUNLPTEAM/class-textbook",split='train')


/Users/chihhlinn/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
print(dataset["type"])

['Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Document', 'Do

In [12]:
def format_func(data):
    return f"text: {data['text']}, metadata: {data['metadata']}, type: {data['type']}, summary: {data['summary']}"

In [24]:
from dotenv import load_dotenv
load_dotenv()

from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_text(format_func(dataset))

# if os.path.exists("./dataset/faiss_db"):
vectorstore = FAISS.from_texts(texts=splits, embedding=OpenAIEmbeddings())
# vectorstore.save_local("faiss_index")
    
# else:
# vectorstore = FAISS.load_local(folder_path= "./database/faiss_db",embeddings=OpenAIEmbeddings, allow_dangerous_deserialization=True)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [25]:
from  langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [28]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [31]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join( doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [32]:
rag_chain.invoke("give me a summary of the text and generate ten questions")

'The text provides an overview of medical terminology, including levels of organization, body parts, and terms of location. It also includes questions and answers that illustrate how biological systems function and interact. Healthcare professionals can benefit from understanding this information to improve patient care.\n\n1. What are the levels of organization of the body from simplest to most complex?\n2. How do negative feedback mechanisms work, and how do they differ from positive feedback mechanisms?\n3. Explain the anatomic position and the anatomic terms for body parts.\n4. Why is it important for healthcare professionals to stay up-to-date with medical terminology and concepts?\n5. What is the significance of understanding metabolism, metabolic rate, and homeostasis in patient care?\n6. How can visualizing the interior of the body help in healthcare practice?\n7. Describe the general plan of the body in terms of organization and structure.\n8. What is the difference between th

In [34]:
str = 'The text provides an overview of medical terminology, including levels of organization, body parts, and terms of location. It also includes questions and answers that illustrate how biological systems function and interact. Healthcare professionals can benefit from understanding this information to improve patient care.\n\n1. What are the levels of organization of the body from simplest to most complex?\n2. How do negative feedback mechanisms work, and how do they differ from positive feedback mechanisms?\n3. Explain the anatomic position and the anatomic terms for body parts.\n4. Why is it important for healthcare professionals to stay up-to-date with medical terminology and concepts?\n5. What is the significance of understanding metabolism, metabolic rate, and homeostasis in patient care?\n6. How can visualizing the interior of the body help in healthcare practice?\n7. Describe the general plan of the body in terms of organization and structure.\n8. What is the difference between the dorsal and ventral body cavities?\n9. How do tissues and organs replace in the body?\n10. Why is it essential for healthcare professionals to know about the terms anatomy, physiology, and pathophysiology?'
print(str)

The text provides an overview of medical terminology, including levels of organization, body parts, and terms of location. It also includes questions and answers that illustrate how biological systems function and interact. Healthcare professionals can benefit from understanding this information to improve patient care.

1. What are the levels of organization of the body from simplest to most complex?
2. How do negative feedback mechanisms work, and how do they differ from positive feedback mechanisms?
3. Explain the anatomic position and the anatomic terms for body parts.
4. Why is it important for healthcare professionals to stay up-to-date with medical terminology and concepts?
5. What is the significance of understanding metabolism, metabolic rate, and homeostasis in patient care?
6. How can visualizing the interior of the body help in healthcare practice?
7. Describe the general plan of the body in terms of organization and structure.
8. What is the difference between the dorsal a

In [36]:
str = rag_chain.invoke("What are the levels of organization of the body from simplest to most complex? (answer in zh-tw)")
print(str)

我不知道。


In [38]:
str = rag_chain.invoke("What are the levels of organization of the body from simplest to most complex?")
print(str)

The levels of organization of the body from simplest to most complex are: chemical, cells, tissues, organs, and organ systems. Each higher level incorporates the structures and functions of the previous level. All levels of organization are depicted in a diagram.


In [40]:
str = rag_chain.invoke("what is Cellular Transport Mechanisms? (answer in zh-tw)")
print(str)
# Cellular transport mechanisms involve processes like diffusion, osmosis, facilitated diffusion, active transport, filtration, phagocytosis, and pinocytosis for moving substances in and out of cells. Centrioles, cilia, and flagella play roles in cell division, cellular movement, and material transport within cells. Errors in DNA can lead to genetic diseases that impact protein synthesis and cellular functions.

我不知道。
